In [1]:
from collections import Counter
from functools import partial, reduce
# from linear_algebra import dot, vector_add
# from gradient_descent import maximize_stochastic, maximize_batch
# from working_with_data import rescale
# from machine_learning import train_test_split
# from multiple_regression import estimate_beta, predict
import math, random

In [2]:
def step(v, direction, step_size):
    return [v_i + step_size * direction_i for v_i, direction_i in zip(v, direction)]

def safe(f):
    def safe_f(*args, **kwargs):
        try:
            return f(*args, **kwargs)
        except:
            return float('inf')
    return safe_f

def negate(f):
    # negates a fnc that returns a value
    return lambda *args, **kwargs: -f(*args, **kwargs)

def negate_all(f):
    # negates a fnc that returns a list
    return lambda *args, **kwargs: [-y for y in f(*args, **kwargs)]

def maximize_batch(target_fn, gradient_fn, theta_0, tolerance=0.000001):
    return minimize_batch(negate(target_fn), negate_all(gradient_fn), theta_0, tolerance)

def minimize_batch(target_fn, gradient_fn, theta_0, tolerance=0.000001):
    step_sizes = [100, 10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001]
    theta = theta_0
    target_fn = safe(target_fn)
    value = target_fn(theta)
    while True:
        gradient = gradient_fn(theta)
        next_thetas = [step(theta, gradient, -step_size) for step_size in step_sizes]
        # choose next theta by minimizing error
        next_theta = min(next_thetas, key=target_fn)
        next_value = target_fn(next_theta)
        # stop if converging
        if abs(value - next_value) < tolerance:
            return theta
        else:
            theta, value = next_theta, next_value
            
            
def split(data, train_fraction):
    results = [], [] # train, test
    for row in data:
        results[0 if random.random() < train_fraction else 1].append(row)
    return results

def train_test_split(x, y, train_fraction=0.66):
    data = zip(x,y)
    train, test = split(data, train_fraction)
    x_train, y_train = list(zip(*train))
    x_test, y_test = list(zip(*test))
    return x_train, y_train, x_test, y_test


In [3]:
def vector_add(v, w):
    """adds two vectors componentwise"""
    return [v_i + w_i for v_i, w_i in zip(v,w)]

def dot(v, w):
    """v_1 * w_1 + ... + v_n * w_n"""
    return sum(v_i * w_i for v_i, w_i in zip(v, w))

def sum_of_squares(v):
    """v_1 * v_1 + ... + v_n * v_n"""
    return dot(v, v)



def de_mean(x):
    """translate x by subtracting its mean (so the result has mean 0)"""
    x_bar = mean(x)
    return [x_i - x_bar for x_i in x]

def mean(x):
    return sum(x) / len(x)

def variance(x):
    """assumes x has at least two elements"""
    n = len(x)
    deviations = de_mean(x)
    return sum_of_squares(deviations) / (n - 1)

def standard_deviation(x):
    return math.sqrt(variance(x))


def shape(A):
    num_rows = len(A)
    num_cols = len(A[0]) if A else 0
    return num_rows, num_cols

def get_row(A, i):
    return A[i]

def get_column(A, j):
    return [A_i[j] for A_i in A]

def make_matrix(num_rows, num_cols, entry_fn):
    """returns a num_rows x num_cols matrix
    whose (i,j)-th entry is entry_fn(i, j)"""
    return [[entry_fn(i, j) for j in range(num_cols)]
            for i in range(num_rows)]


def scale(data_matrix):
    num_rows, num_cols = shape(data_matrix)
    means = [mean(get_column(data_matrix,j))
             for j in range(num_cols)]
    stdevs = [standard_deviation(get_column(data_matrix,j))
              for j in range(num_cols)]
    return means, stdevs

def rescale(data_matrix):
    """rescales the input data so that each column
    has mean 0 and standard deviation 1
    ignores columns with no deviation"""
    means, stdevs = scale(data_matrix)

    def rescaled(i, j):
        if stdevs[j] > 0:
            return (data_matrix[i][j] - means[j]) / stdevs[j]
        else:
            return data_matrix[i][j]

    num_rows, num_cols = shape(data_matrix)
    return make_matrix(num_rows, num_cols, rescaled)


In [4]:
def logistic(x):
    return 1.0 / (1 + math.exp(-x))

def logistic_prime(x):
    return logistic(x) * (1 - logistic(x))

def logistic_log_likelihood_i(x_i, y_i, beta):
    if y_i == 1:
        return math.log(logistic(dot(x_i, beta)))
    else:
        return math.log(1 - logistic(dot(x_i, beta)))

def logistic_log_likelihood(x, y, beta):
    return sum(logistic_log_likelihood_i(x_i, y_i, beta)
               for x_i, y_i in zip(x, y))

def logistic_log_partial_ij(x_i, y_i, beta, j):
    """here i is the index of the data point,
    j the index of the derivative"""

    return (y_i - logistic(dot(x_i, beta))) * x_i[j]

def logistic_log_gradient_i(x_i, y_i, beta):
    """the gradient of the log likelihood
    corresponding to the i-th data point"""

    return [logistic_log_partial_ij(x_i, y_i, beta, j)
            for j, _ in enumerate(beta)]

def logistic_log_gradient(x, y, beta):
    return reduce(vector_add,
                  [logistic_log_gradient_i(x_i, y_i, beta)
                   for x_i, y_i in zip(x,y)])

In [5]:
import logreg_inputs

# each element is [1, experience, salary]
data = logreg_inputs.DATA
x = [[1] + row[:2] for row in data]
y = [row[2] for row in data]
rescaled_x = rescale(x)

random.seed(0)
x_train, y_train, x_test, y_test = train_test_split(rescaled_x, y, train_fraction=0.66)

fn = partial(logistic_log_likelihood, x_train, y_train)
grad_fn = partial(logistic_log_gradient, x_train, y_train)

In [6]:
# pick a random starting point
beta_0 = [random.random() for _ in range(3)]

# and maximize using gradient descent
beta_hat = maximize_batch(fn, grad_fn, beta_0)

print("beta_0", beta_0)
print("beta_batch", beta_hat)

beta_0 [0.44535218971882984, 0.25922692344722276, 0.15768657212231085]
beta_batch [-1.8935557020921057, 4.0148875697806305, -3.8443576079115345]


In [7]:
def fuzz(k):
    return k * random.gauss(0,1)

def dot(v1, v2):
    return sum([(v1i*v2i) for v1i, v2i in zip(v1, v2)])

def vector_add(v1, v2):
    return [(v1_i + v2_i) for v1_i, v2_i in zip(v1, v2)]

def logistic(a):
    return 1 / (1 + math.exp(-a))

def make_data(true_params, n=1000, k=0.7):
    X = [[random.uniform(-3,3) for _ in range(len(true_params)-1)] + [1] for _ in range(n)]
    N = [dot(x, true_params) for x in X]
    S = [logistic(n) + fuzz(k) for n in N]
    Y = [1 if random.random() <= s else 0 for s in S]
    return X, Y


In [8]:
#random.seed(0)
beta_true = [4, -3, -1]
beta_0 = [random.random() for _ in range(len(beta_true))]

x, y = make_data(beta_true)

rescaled_x = rescale(x)

x_train, y_train, x_test, y_test = train_test_split(rescaled_x, y, train_fraction=0.66)

fn = partial(logistic_log_likelihood, x_train, y_train)
grad_fn = partial(logistic_log_gradient, x_train, y_train)

In [9]:
# and maximize using gradient descent
beta_hat = maximize_batch(fn, grad_fn, beta_0)

print("beta_batch", beta_hat)

beta_batch [0.7057577613714192, -0.40935071705217896, -0.13842910852034962]


In [10]:
[(beta_true[0] / beta_hat[0]) * bi for bi in beta_hat]

[4.0, -2.3200635654745563, -0.7845700952766342]